In [25]:
import pandas as pd
import requests
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
from dotenv import load_dotenv 

import csv
import json

In [26]:
import time #para el time sleep 

Ejemplo de endpoint: "https://musicbrainz.org/ws/2/artist/?query=artist:XXXXXXXXX&fmt=json"

Función llamar api

In [22]:
def llamar_api(url):

    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.
    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  # Imprime el código de estado de la respuesta.
    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
        print(f"El motivo por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
    else:
        return llamada.json()  # Si la llamada fue exitosa, devuelve los datos de respuesta en formato JSON.

Sacar los artistas de Spotify: 

In [ ]:
# Paso 1: Leer el archivo CSV
spoti_csv_file = "../data/dataframes_combinados.csv" # nombre de tu archivo Spoti CSV
spoti_json_file = "dataframes_combinados.json"  # nombre del archivo Spoti de salida JSON

# Paso 2: Abrir el CSV y convertir a formato JSON
with open(spoti_csv_file, mode='r', encoding='utf-8') as file:
    # Leer el contenido CSV
    csv_reader = csv.DictReader(file)
    
    # Convertir cada fila a diccionario y agregarla a una lista
    data = [row for row in csv_reader]

# Paso 3: Escribir los datos en un archivo JSON
with open(spoti_json_file, mode='w', encoding='utf-8') as file:
    json.dump(data, file, indent=4)  # Guarda el JSON con formato legible (indent=4)

print(f'Archivo {spoti_json_file} creado con éxito.')

In [11]:
#Traemos el archivo a json
with open("dataframes_combinados.json","r") as f: 
    result_spoti = json.load(f)

In [ ]:
result_spoti

In [12]:
#Creamos lista vacia para que se complete con los artistas
artist = []

for item in result_spoti: 
    artist.append(item['artista'])

In [ ]:
#Convertimos en set para eliminar artistas duplicados (hay artistas con mas de una canción en el mismo año)
set_artist = set(artist)
set_artist

In [15]:
list_artistas = list(set_artist)
len(list_artistas)

893

In [ ]:
list_artistas

## Probando cambios en dict_artistas["fecha_nacimiento"] & dict_artistas["fecha_inicio_actividad"]: 

In [ ]:
 

# Lista de artistas de spotify despues de eliminar duplicados
list_artistas

# Dict donde se almacenara la info de los artistas de la API musicBrainz
dict_artistas = {"artista" : [], "pais_de_origen" : [], "area_de_origen" : [], "fecha_nacimiento" : [], "fecha_inicio_actividad" : [], "fecha_fin_actividad" :[]}

for artista in list_artistas:
    
    # Protegemos posibles errores al hacer la llamada a la URL para cada artista
    try:
        # Construir la URL con el nombre del artista
        url = f"https://musicbrainz.org/ws/2/artist/?query=artist:{artista}&fmt=json"
        # Petición a la API con funcion llamar_api definida anteriormente -> devuelve resultado en JSON
        result = llamar_api(url)
            
        # Comprobamos que en el dict que nos da la URL por cada artista tiene el key artist y que el nombre del primer artista de la lista no esta vacio  
        if result.get("artists") and result["artists"][0].get("name") is not None:
        
            # Cogemos el primer dict de la lista artists porque hemos hecho la hipotesis de que es el mas correcto
            primer_artista_dict = result["artists"][0]    
            
            # Rellenamos el dict general con toda la info de los artistas
            try:
                dict_artistas["artista"].append(primer_artista_dict["name"])
            except KeyError:
                dict_artistas["artista"].append("NULL")
                
            try:
                dict_artistas["pais_de_origen"].append(primer_artista_dict["area"]["name"])
            except KeyError:
                dict_artistas["pais_de_origen"].append("NULL")  
                
            try:
                dict_artistas["area_de_origen"].append(primer_artista_dict["begin-area"]["name"])
            except KeyError:
                dict_artistas["area_de_origen"].append("NULL")  
        
            # Aqui ponemos un get para los casos en los que falte el life-span (que devuelva un diccionario vacio en caso que no esté presente)
           
            try:
                if primer_artista_dict.get("type") == "Person":
                    fecha_nacimiento = primer_artista_dict.get("life-span", {}).get("begin", "NULL")
                    dict_artistas["fecha_nacimiento"].append(fecha_nacimiento)
                    dict_artistas["fecha_inicio_actividad"].append("NULL")
                elif primer_artista_dict.get("type") == "Group":
                    dict_artistas["fecha_nacimiento"].append("NULL")
                    fecha_inicio = primer_artista_dict.get("life-span", {}).get("begin", "NULL")
                    dict_artistas["fecha_inicio_actividad"].append(fecha_inicio)
                else:
                    dict_artistas["fecha_nacimiento"].append("NULL")
                    dict_artistas["fecha_inicio_actividad"].append("NULL")  # Agregar NULL si es otro tipo
            except KeyError:
                dict_artistas["fecha_nacimiento"].append("NULL")
                dict_artistas["fecha_inicio_actividad"].append("NULL")
            
            try:
                if primer_artista_dict["life-span"].get("ended") is not None: #ver que ended este presente
                    if primer_artista_dict["life-span"]["ended"]: #Verificar si ended es True
                        fecha_fin = primer_artista_dict["life-span"].get("end", "No activo") #creamos variable para luego imprimir fecha fin
                        dict_artistas["fecha_fin_actividad"].append(fecha_fin)     
                    else: 
                        dict_artistas["fecha_fin_actividad"].append("Actualidad") 
                else:
                    dict_artistas["fecha_fin_actividad"].append("Actualidad")
            except KeyError:
                dict_artistas["fecha_fin_actividad"].append("NULL")
                         
    except requests.exceptions.RequestException as e:
        # Capturar errores de red (e.g., problemas con la conexión)
        print(f"Error al hacer la solicitud para {artista}: {e}")
    
    except ValueError as e:
        # Capturar errores de JSON (si la respuesta no es JSON válido)
        print(f"Error al procesar la respuesta JSON para {artista}: {e}")
    
    except Exception as e:
        # Capturar cualquier otro tipo de error no esperado
        print(f"Algo salió mal con {artista}: {e}")

    # Pausar de 1 segundos entre cada solicitud para evitar ser bloqueado por la API
    time.sleep(1)
    

In [52]:
for key, value in dict_artistas.items():
    print(f"{key}: {len(value)}")

artista: 884
pais_de_origen: 884
area_de_origen: 884
fecha_nacimiento: 884
fecha_inicio_actividad: 884
fecha_fin_actividad: 884


In [51]:
dict_artistas = pd.DataFrame(dict_artistas) 
dict_artistas

,artista,pais_de_origen,area_de_origen,fecha_nacimiento,fecha_inicio_actividad,fecha_fin_actividad
0,NEEDTOBREATHE,United States,Seneca,NULL,2000,Actualidad
1,David Bisbal,Spain,Almería,1979-06-05,NULL,Actualidad
2,ELYELLA,Madrid,NULL,NULL,NULL,Actualidad
3,Floating Points,United Kingdom,Manchester,NULL,NULL,Actualidad
4,Los Fabulosos Cadillacs,Argentina,Buenos Aires,NULL,1985-01-19,Actualidad
5,Old Dominion,United States,Nashville,NULL,2007,Actualidad
6,Bloc Party,United Kingdom,Reading,NULL,1999-08,Actualidad
7,Mónica Naranjo,Spain,Figueres,1974-05-23,NULL,Actualidad
8,La Pegatina,Spain,Montcada i Reixac,NULL,2003,Actualidad
9,The Blue Stones,Canada,Windsor,NULL,2011,Actualidad
